# Stage 1 - Scraping Lift Off!

In [29]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pymongo
import pandas as pd

In [30]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\willt\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


## NASA Mars News

In [31]:
# Source Scrape URL 
news_url = 'https://redplanetscience.com/'
browser.visit(news_url)

In [32]:
# Retrieve page
news_retrieve = requests.get(news_url)

# Create BeautifulSoup object
news_soup = BeautifulSoup(news_retrieve.text, 'html.parser')

news_html = browser.html
news_results = BeautifulSoup(news_html, 'html.parser')

In [33]:
# Retrieve headline and flavor text
news_title = news_results.find('div', class_='content_title').get_text()
news_teaser = news_results.find('div', class_='article_teaser_body').get_text()

print(news_title)
print(news_teaser)

The MarCO Mission Comes to an End
The pair of briefcase-sized satellites made history when they sailed past Mars in 2019.


## JPL Mars Space Images - Featured Image

In [34]:
# Source Scrape URL 
images_url = 'https://spaceimages-mars.com/'
browser.visit(images_url)

In [35]:
# Retrieve page
images_retrieve = requests.get(images_url)

# Create BeautifulSoup object
images_soup = BeautifulSoup(images_retrieve.text, 'html.parser')

images_html = browser.html
images_results = BeautifulSoup(images_html, 'html.parser')

In [36]:
# Retrieve Current Header Image from URL 
featured_image = images_results.find('img', class_='headerimage')
featured_image = featured_image['src']

print(featured_image)

image/featured/mars3.jpg


## Mars Facts

In [37]:
# Source Scrape URL 
facts_url = 'https://galaxyfacts-mars.com/'

In [38]:
tables = pd.read_html(facts_url)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [39]:
#Set table data into Dataframe
facts_df = tables[0]
facts_df.head(20)

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [40]:
#Renamed Columns
facts_df.rename(columns={0:'Index', 1: 'Mars Value', 2: 'Earth Values'})

,Index,Mars Value,Earth Values
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [41]:
#Converted DataFrame into HTML string
facts_html = facts_df.to_html()
print(facts_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
      <th>2</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Mars - Earth Comparison</td>
      <td>Mars</td>
      <td>Earth</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Diameter:</td>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Distance from Sun:</td>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Length of Year:</td>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Temperature:</td>
      <td>-87 to -5 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody

## Mars Hemishperes

In [44]:
# Source Scrape URL 
hemispheres_url = 'https://marshemispheres.com/'

In [45]:
# Retrieve page
hemispheres_retrieve = requests.get(hemispheres_url)

# Create BeautifulSoup object
hemi_soup = BeautifulSoup(hemispheres_retrieve.text, 'html.parser')

hemispheres_html = browser.html
hemispheres_results = BeautifulSoup(hemispheres_html, 'html.parser')

In [47]:
# Get URL of pages to be scraped
url_results = hemi_soup.find_all('div', class_='item')
url_list=[]

for result in url_results:
    hemisphere=result.find('a')['href']
    url_list.append(hemisphere)

print(url_list)

['cerberus.html', 'schiaparelli.html', 'syrtis.html', 'valles.html']


In [50]:
#Link Titles and image URLs
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "valles.html"},
    {"title": "Cerberus Hemisphere", "img_url": "cerberus.html"},
    {"title": "Schiaparelli Hemisphere", "img_url": "schiaparelli.html"},
    {"title": "Syrtis Major Hemisphere", "img_url": "syrtis.html"},
]

In [51]:
browser.quit()